In [1]:
import os
import glob
import random
import torchvision
import torch
import cv2
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import timm

from albumentations import *
from albumentations.pytorch import ToTensorV2
import torch
import torch.utils.data as data
from PIL import Image
from adamp import AdamP
from adamp import SGDP
from tqdm.notebook import tqdm

In [2]:
!ls

004_loss_0.017.ckpt  8.30_1  8.30_3	       version3.ipynb  vgg19
8.29		     8.30_2  requirements.txt  version4.ipynb


In [3]:
TAR_PATH = '/opt/ml/input/data/train'
PARENT_DATA_PATH = '/opt/ml/input'
DATA_PATH = os.path.join(PARENT_DATA_PATH, "data")
DATA_PATH

'/opt/ml/input/data'

In [4]:
TRAIN_IMGS_DATASET_PATH = "/opt/ml/input/data/train/images"
EVAL_IMGS_DATASET_PATH = "/opt/ml/input/data/eval/images"

list_individuals = glob.glob(os.path.join(TRAIN_IMGS_DATASET_PATH, "*"))
print(len(list_individuals) ,list_individuals[:10])

2700 ['/opt/ml/input/data/train/images/000009_female_Asian_56', '/opt/ml/input/data/train/images/000574_female_Asian_56', '/opt/ml/input/data/train/images/000041_female_Asian_58', '/opt/ml/input/data/train/images/004488_male_Asian_23', '/opt/ml/input/data/train/images/003880_female_Asian_55', '/opt/ml/input/data/train/images/001617_female_Asian_37', '/opt/ml/input/data/train/images/001047_male_Asian_60', '/opt/ml/input/data/train/images/003567_female_Asian_25', '/opt/ml/input/data/train/images/001308_male_Asian_26', '/opt/ml/input/data/train/images/001751_male_Asian_51']


In [5]:
len(os.listdir(TRAIN_IMGS_DATASET_PATH))

2700

In [6]:
superflous_folders = glob.glob(os.path.join(TRAIN_IMGS_DATASET_PATH,"._*"))
print(len(superflous_folders),superflous_folders[:10])
for item in superflous_folders:
  os.remove(item)

0 []


In [7]:
personal_id = "001771_female_Asian_54"
os.listdir(os.path.join(TRAIN_IMGS_DATASET_PATH, personal_id))

['mask4.jpg',
 'mask1.jpg',
 'incorrect_mask.jpg',
 'mask3.jpg',
 'mask5.jpg',
 'normal.jpg',
 'mask2.jpg']

In [8]:
list_folders = glob.glob(os.path.join(TRAIN_IMGS_DATASET_PATH,"*"))
for folder in list_folders:
  superflous_files = glob.glob(os.path.join(folder,"._*"))
  for file in superflous_files:
    os.remove(file)
    
personal_id = "001771_female_Asian_54"
os.listdir(os.path.join(TRAIN_IMGS_DATASET_PATH, personal_id))

['mask4.jpg',
 'mask1.jpg',
 'incorrect_mask.jpg',
 'mask3.jpg',
 'mask5.jpg',
 'normal.jpg',
 'mask2.jpg']

In [9]:
for individual_name in list_individuals:
  individual_path = os.path.join(TRAIN_IMGS_DATASET_PATH, individual_name)
  
  # check whether dataset is properly untarred
  jpg_file_list = glob.glob(f"{individual_path}/*.jpg")
  jpeg_file_list = glob.glob(f"{individual_path}/*.jpeg")
  png_file_list = glob.glob(f"{individual_path}/*.png")
  list_images = jpg_file_list + jpeg_file_list + png_file_list
  num_jpg_images = len(list_images)
  # print(num_jpg_images)
  if num_jpg_images == 7:
    pass
  else:
    print(f"{individual_path} has a problem during untar process, only has {num_jpg_images} images")
    pass
print("process cleared, untarred properly")

process cleared, untarred properly


In [10]:
!pip install adamp

In [11]:
!pip install timm

In [12]:

t_transform = Compose([
    #Resize(img_size[0], img_size[1], p=1.0),
    #Resize(200, 260, p=1.0),
    CenterCrop(height = 400, width = 250), # add centercrop 350/350 -> 400/200 -> 300/300
    #HorizontalFlip(p=0.5),
    #ShiftScaleRotate(p=0.5),
    #HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
    RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    #GaussNoise(p=0.5),
    Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), max_pixel_value=255.0, p=1.0),
    ToTensorV2(p=1.0),
    ], p=1.0)


v_transform = Compose([
    #Resize(img_size[0], img_size[1]),
    #Resize(200, 260),
    CenterCrop(height = 400, width = 250), # add centercrop
    Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), max_pixel_value=255.0, p=1.0),
    ToTensorV2(p=1.0),
    ], p=1.0)

In [13]:
class MaskLabels:
    mask = 0
    incorrect = 1
    normal = 2

class GenderLabels:
    male = 0
    female = 1

class AgeGroup:
    map_label = lambda x: 0 if int(x) < 30 else 1 if int(x) < 58 else 2

In [14]:
class MaskBaseDataset(data.Dataset):
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1.jpg": MaskLabels.mask,
        "mask2.jpg": MaskLabels.mask,
        "mask3.jpg": MaskLabels.mask,
        "mask4.jpg": MaskLabels.mask,
        "mask5.jpg": MaskLabels.mask,
        "incorrect_mask.jpg": MaskLabels.incorrect,
        "normal.jpg": MaskLabels.normal,
        
        "mask1.jpeg": MaskLabels.mask,
        "mask2.jpeg": MaskLabels.mask,
        "mask3.jpeg": MaskLabels.mask,
        "mask4.jpeg": MaskLabels.mask,
        "mask5.jpeg": MaskLabels.mask,
        "incorrect_mask.jpeg": MaskLabels.incorrect,
        "normal.jpeg": MaskLabels.normal,
        
        "mask1.png": MaskLabels.mask,
        "mask2.png": MaskLabels.mask,
        "mask3.png": MaskLabels.mask,
        "mask4.png": MaskLabels.mask,
        "mask5.png": MaskLabels.mask,
        "incorrect_mask.png": MaskLabels.incorrect,
        "normal.png": MaskLabels.normal
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, img_dir, mean, std, transform=None):
        """
        MaskBaseDataset을 initialize 합니다.

        Args:
            img_dir: 학습 이미지 폴더의 root directory 입니다.
            transform: Augmentation을 하는 함수입니다.
        """
        self.img_dir = img_dir
        self.mean = mean
        self.std = std
        self.transform = transform

        self.setup()

    def set_transform(self, transform):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform = transform
        
    def setup(self):
        """
        image의 경로와 각 이미지들의 label을 계산하여 저장해두는 함수입니다.
        """
        profiles = os.listdir(self.img_dir)
        for profile in profiles:
            for file_name, label in self._file_names.items():
                img_path = os.path.join(self.img_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                if os.path.exists(img_path):
                    self.image_paths.append(img_path)
                    self.mask_labels.append(label)

                    id, gender, race, age = profile.split("_")
                    gender_label = getattr(GenderLabels, gender)
                    age_label = AgeGroup.map_label(age)

                    self.gender_labels.append(gender_label)
                    self.age_labels.append(age_label)

    def __getitem__(self, index):
        """
        데이터를 불러오는 함수입니다. 
        데이터셋 class에 데이터 정보가 저장되어 있고, index를 통해 해당 위치에 있는 데이터 정보를 불러옵니다.
        
        Args:
            index: 불러올 데이터의 인덱스값입니다.
        """
        # 이미지를 불러옵니다.
        image_path = self.image_paths[index]
        image = Image.open(image_path)
        
        # 레이블을 불러옵니다.
        mask_label = self.mask_labels[index]
        gender_label = self.gender_labels[index]
        age_label = self.age_labels[index]
        multi_class_label = mask_label * 6 + gender_label * 3 + age_label
        
        # 이미지를 Augmentation 시킵니다.
        image_transform = self.transform(image=np.array(image))['image']
        return image_transform, multi_class_label

    def __len__(self):
        return len(self.image_paths)

In [15]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

In [16]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=3, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))


In [17]:
class F1Loss(nn.Module):
    def __init__(self, classes=3, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon
    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()


In [18]:
_criterion_entrypoints = {
    'cross_entropy': nn.CrossEntropyLoss,
    'focal': FocalLoss,
    'label_smoothing': LabelSmoothingLoss,
    'f1': F1Loss
}

def criterion_entrypoint(criterion_name):
    return _criterion_entrypoints[criterion_name]


def is_criterion(criterion_name):
    return criterion_name in _criterion_entrypoints


def create_criterion(criterion_name, **kwargs):
    if is_criterion(criterion_name):
        create_fn = criterion_entrypoint(criterion_name)
        criterion = create_fn(**kwargs)
    else:
        raise RuntimeError('Unknown loss (%s)' % criterion_name)
    return criterion


In [19]:
def f1_loss(y_true:torch.Tensor, y_pred:torch.Tensor, is_training=False) -> torch.Tensor:
    
    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
        
    
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    return f1

In [20]:
RGB_Mean = [0.56019358, 0.52410121, 0.501457]
RGB_SD = [0.23318603, 0.24300033, 0.24567522]

In [21]:
BASE_DIR = '/opt/ml/input/data/train'
IMG_DIR = f'{BASE_DIR}/images'
TRAIN_CSV = f'{BASE_DIR}/train.csv'
SPLIT_RATIO = 0.2
TRAIN_BATCH_SIZE = 32
VAL_BATCH_SIZE = 32

In [22]:
#transform_t = get_train_transforms(mean=RGB_Mean, std=RGB_SD)
#transform_v = get_val_transforms(mean=RGB_Mean, std=RGB_SD)
dataset = MaskBaseDataset(
    img_dir=IMG_DIR,
    mean = RGB_Mean,
    std = RGB_SD
)

# train dataset과 validation dataset을 9:1 비율로 나눕니다.
n_val = int(len(dataset) * SPLIT_RATIO)
n_train = len(dataset) - n_val
train_dataset, val_dataset = data.random_split(dataset, [n_train, n_val])

# 각 dataset에 augmentation 함수를 설정합니다.
train_dataset.dataset.set_transform(t_transform)
val_dataset.dataset.set_transform(v_transform)
print(train_dataset)
print(val_dataset)

In [23]:
#train_dataset = CutMix(train_dataset, num_class=1000, beta=1.0, prob=0.5, num_mix=2)
#val_dataset = CutMix(val_dataset, num_class=1000, beta=1.0, prob=0.5, num_mix=2)

In [24]:
train_loader = data.DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    num_workers=2,
    shuffle=True
)

valid_loader = data.DataLoader(
    val_dataset,
    batch_size=VAL_BATCH_SIZE,
    num_workers=2,
    shuffle=False
)

In [25]:
images, labels = next(iter(train_loader))
print(f'images shape: {images.shape}')
print(f'labels shape: {labels.shape}')

images shape: torch.Size([32, 3, 400, 250])
labels shape: torch.Size([32])


In [26]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('GPU Name:', torch.cuda.get_device_name(0))
else:
    print('No GPU, using CPU.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
GPU Name: Tesla V100-PCIE-32GB


In [27]:
EPOCHS = 10
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-5

In [28]:
from efficientnet_pytorch import EfficientNet
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 18):
        super(MyModel, self).__init__()
        # Transfer learning to add final layers in the end.
        # Model Comaprison: https://paperswithcode.com/sota/image-classification-on-imagenet
        # self.backbone = models.resnet50(pretrained=True)
        # self.backbone = models.resnext50_32x4d(pretrained=True)
        
        model_architecture = 'tf_efficientnet_b4'
        self.backbone = timm.create_model(model_architecture, pretrained=True)
        n_features = self.backbone.classifier.in_features
        print(n_features)
        self.backbone.fc = nn.Linear(in_features=n_features, out_features=num_classes, bias=True)
        #self.backbone.fc = nn.Linear(in_features=2048, out_features=18, bias=True)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.backbone(x)
        return x

In [29]:
model = MyModel()
model.cuda()

1792


MyModel(
  (backbone): EfficientNet(
    (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_run

In [30]:
from adamp import AdamP


# # Freeze the feature extracting convolution layers
# for param in model.features.parameters():
#     param.requires_grad = False

criterion = FocalLoss(gamma = 5)
#criterion = CutMixCrossEntropyLoss(True)
# criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
optimizer = AdamP(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999), weight_decay= WEIGHT_DECAY)
#optimizer = SGDP(model.parameters(), lr=LEARNING_RATE, weight_decay= WEIGHT_DECAY,momentum=0.9, nesterov=True)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

In [31]:
class Metrics(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [32]:
from tqdm.notebook import tqdm

def train(model, epochs, train_loader, valid_loader, save_path):
    best_valid_acc = 0
    best_valid_loss = 100000

    for epoch in range(epochs):
        epoch_train_loss, epoch_train_acc = Metrics(), Metrics()
        early_stop = 0
        for iter, (img,label) in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()

            img, label = img.type(torch.FloatTensor).to(device), label.to(device)

            # 모델에 이미지 forward
            model.train()
            pred_logit = model(img)

            # loss 값 계산
            loss = criterion(pred_logit,label)

            # Backpropagation
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Accuracy 계산
            pred_label = torch.max(pred_logit.data,1)

            # pred_label  = pred_logit.argmax(1)
            acc = (pred_label.indices == label).sum().item() / len(label)

            epoch_train_loss.update(loss.item(), len(img))
            epoch_train_acc.update(acc, len(img))

            train_loss = loss.item()
            train_acc = acc
            
#             if (iter % 300 == 0) or (iter == len(train_loader)-1):
#                 print("Iter [%3d/%3d] | Train Loss %.4f | Train Acc %.4f" %(iter, len(train_loader), train_loss, train_acc))

        epoch_train_loss = epoch_train_loss.avg
        epoch_train_acc = epoch_train_acc.avg
        print("Epoch %d | Train Loss %.4f | Train Acc %.4f"%(epoch,epoch_train_loss, epoch_train_acc))
        
        valid_loss, valid_acc, valid_f1 = Metrics(), Metrics(), Metrics()
        model.eval()
        
        for img, label in valid_loader:
            img, label = img.type(torch.FloatTensor).to(device), label.to(device)

            with torch.no_grad():
                pred_logit = model(img)
            loss = criterion(pred_logit, label) 
            pred_label = torch.max(pred_logit.data,1)

            acc = (pred_label.indices == label).sum().item() / len(label)

            valid_loss.update(loss.item(), len(img))
            valid_acc.update(acc, len(img))
            valid_f1.update(f1_loss(label, pred_label.indices), len(img))


        valid_loss = valid_loss.avg
        valid_acc = valid_acc.avg
        valid_f1 = valid_f1.avg
        print("Valid Loss %.4f | Valid Acc %.4f | f1 score %.4f" %(valid_loss, valid_acc, valid_f1))
        
        if valid_loss <= best_valid_loss:
            print("New valid model for val loss! saving the model...")
            torch.save(model.state_dict(),PATH + f"{epoch:03}_loss_{valid_loss:4.2}.ckpt")
            best_valid_loss = valid_loss
            early_stop = 0
        else:
            early_stop += 1
            if early_stop > 2:
                break
        
        if valid_acc > best_valid_acc:
            print("New valid model for val accuracy! saving the model...")
            torch.save(model.state_dict(),PATH + f"{epoch:03}_loss_{valid_loss:4.2}.ckpt")
            best_valid_acc = valid_acc

In [33]:
PATH = "./"

# (Train Image Size = 1890 * 9) / (Batch Size = 32) = 532 as Length
train(model, EPOCHS, train_loader, valid_loader, PATH)


Epoch 0 | Train Loss 0.2437 | Train Acc 0.7788


KeyboardInterrupt: 

In [ ]:
# EVAL

In [ ]:
class TestDataset(data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image, self.img_paths[index]

    def __len__(self):
        return len(self.img_paths)

In [ ]:
model_path = "/opt/ml/code/004_loss_0.017.ckpt"
model = MyModel()
model.load_state_dict(torch.load(model_path))
model.cuda()

In [ ]:
PRETRAINED_MODEL_NAME = "eff_b4_v4"

In [ ]:
! pip install ttach

In [ ]:
import pandas as pd
import numpy as np
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import ttach as tta
TEST_DIR = "/opt/ml/input/data/eval"

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(TEST_DIR, 'info.csv'))
image_dir = os.path.join(TEST_DIR, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

valid_transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    transforms.CenterCrop((400,200)),
    ToTensor(),
    transforms.Normalize((0.56019358, 0.52410121, 0.501457), (0.23318603, 0.24300033, 0.24567522)),
])
'''
valid_transform = tta.Compose([
    #Resize((512, 384), Image.BILINEAR),
    #tta.CenterCrop(400,200),
    #tta.ToTensor(),
    #tta.Normalize((0.56019358, 0.52410121, 0.501457), (0.23318603, 0.24300033, 0.24567522)),
    tta.HorizontalFlip(),
    tta.Rotate90(angles=[0, 180]),
    tta.Scale(scales=[1, 2, 4]),
    tta.Multiply(factors=[0.9, 1, 1.1]),    
])
'''
dataset = TestDataset(image_paths, valid_transform)

loader = data.DataLoader(
    dataset,
    shuffle=False
)

model.eval()

pred_result = []

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images, path in tqdm(loader):
    temp = []
    temp.append(path)
    with torch.no_grad():
        images = images.type(torch.FloatTensor).to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        temp.append(pred)
        all_predictions.extend(pred.cpu().numpy())
    pred_result.append(temp)
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
file_name = f"{PRETRAINED_MODEL_NAME}_submission.csv"
submission.to_csv(os.path.join(TEST_DIR, file_name), index=False)
print('test inference is done!')